# Function 1: Load and Explore Raster Data 🗺️

**Welcome to your first rasterio function!**

In this notebook, you'll learn how to build the `load_and_explore_raster()` function step by step. This is like opening a GeoTIFF file in QGIS and checking its properties - but doing it programmatically with Python.

## 🎯 What This Function Does
- Opens a raster file (GeoTIFF, NetCDF, etc.) safely using rasterio
- Extracts essential metadata: dimensions, number of bands, coordinate system
- Gets spatial information: extent, pixel size, geographic bounds
- Displays a clear summary of the raster properties
- Handles errors gracefully (missing files, corrupt data)
- Returns a structured dictionary with all the key information

## 🔧 Function Signature
```python
def load_and_explore_raster(raster_path):
    """
    Args:
        raster_path (str): Path to raster file (e.g., 'data/elevation_dem.tif')
    
    Returns:
        dict: Dictionary containing raster properties and metadata
    """
```

## 🚀 Step 1: Import Required Libraries

First, let's import the libraries we need for raster processing:

In [ ]:
import rasterio
import numpy as np
from pathlib import Path
import os

print(f"✅ Rasterio version: {rasterio.__version__}")
print(f"✅ NumPy version: {np.__version__}")
print("🗺️ Ready to work with raster data!")

## 📁 Step 2: Understanding Our Sample Data

Before we load raster data, let's see what files we have available:

In [ ]:
# Let's see what files we have in the data directory
data_dir = Path('../data')  # Go up one level from notebooks, then into data

if data_dir.exists():
    print("📂 Files in data directory:")
    for file in data_dir.iterdir():
        if file.is_file():
            print(f"   📄 {file.name} ({file.stat().st_size / 1024:.1f} KB)")
else:
    print(f"❌ Directory not found: {data_dir}")
    print(f"Current working directory: {Path.cwd()}")

## 🗺️ Step 3: Your First Raster File

Let's start by opening our elevation data and seeing what rasterio tells us about it:

In [ ]:
# Define the file path
elevation_file = data_dir / 'elevation_dem.tif'

print(f"🎯 Loading raster file: {elevation_file}")
print(f"📁 File exists: {elevation_file.exists()}")

if elevation_file.exists():
    print(f"📊 File size: {elevation_file.stat().st_size / 1024:.1f} KB")
else:
    print("❌ Elevation file not found. Check your data directory.")

## 🔓 Step 4: Opening a Raster File Safely

**Key Concept**: Always use a context manager (`with` statement) when working with rasterio files. This ensures the file is properly closed even if an error occurs.

In [ ]:
# Open the raster file using a context manager
try:
    with rasterio.open(elevation_file) as src:
        print("✅ Successfully opened the raster file!")
        print(f"📏 Dimensions: {src.width} x {src.height} pixels")
        print(f"📊 Number of bands: {src.count}")
        print(f"🗂️ Data type: {src.dtypes[0]}")
        print(f"📐 Coordinate Reference System: {src.crs}")
        
except rasterio.RasterioIOError as e:
    print(f"❌ Error opening file: {e}")
except FileNotFoundError:
    print(f"❌ File not found: {elevation_file}")

## 📊 Step 5: Extracting Basic Metadata

Let's dive deeper into the metadata that rasterio provides:

In [ ]:
# Extract comprehensive metadata
with rasterio.open(elevation_file) as src:
    print("=== BASIC RASTER INFORMATION ===")
    print(f"Width (columns): {src.width} pixels")
    print(f"Height (rows): {src.height} pixels")
    print(f"Number of bands: {src.count}")
    print(f"Data type: {src.dtypes[0]}")
    print(f"Driver (format): {src.driver}")
    
    print("\n=== SPATIAL INFORMATION ===")
    print(f"Coordinate Reference System: {src.crs}")
    print(f"NoData value: {src.nodata}")
    
    # Get the spatial transform (how pixels map to coordinates)
    transform = src.transform
    print(f"Pixel size X: {transform.a} degrees")
    print(f"Pixel size Y: {abs(transform.e)} degrees")  # abs() because Y is typically negative

## 🌍 Step 6: Understanding Spatial Extent

The spatial extent tells us the geographic boundaries of our raster:

In [ ]:
# Get the geographic extent (bounding box)
with rasterio.open(elevation_file) as src:
    bounds = src.bounds
    
    print("=== GEOGRAPHIC EXTENT ===")
    print(f"Left (West): {bounds.left:.6f}")
    print(f"Bottom (South): {bounds.bottom:.6f}")
    print(f"Right (East): {bounds.right:.6f}")
    print(f"Top (North): {bounds.top:.6f}")
    
    # Calculate the width and height in coordinate units
    width_coords = bounds.right - bounds.left
    height_coords = bounds.top - bounds.bottom
    
    print(f"\n=== EXTENT DIMENSIONS ===")
    print(f"Width in coordinate units: {width_coords:.6f}")
    print(f"Height in coordinate units: {height_coords:.6f}")
    
    # If this is in degrees, we can give a rough distance estimate
    if str(src.crs).startswith('EPSG:4326') or 'WGS84' in str(src.crs):
        # Very rough approximation: 1 degree ≈ 111 km at equator
        print(f"\n=== APPROXIMATE REAL-WORLD SIZE ===")
        print(f"Width: ~{width_coords * 111:.1f} km")
        print(f"Height: ~{height_coords * 111:.1f} km")
        print("(Note: This is a rough approximation for WGS84 coordinates)")

## 🔍 Step 7: Creating a Comprehensive Summary Function

Now let's put it all together in a function that extracts and organizes all this information:

In [ ]:
def explore_raster_detailed(raster_path):
    """
    Detailed exploration of a raster file - this is similar to what
    your assignment function should do, but more verbose for learning.
    """
    try:
        with rasterio.open(raster_path) as src:
            # Basic properties
            basic_info = {
                'width': src.width,
                'height': src.height,
                'count': src.count,
                'dtype': str(src.dtypes[0]),
                'driver': src.driver,
                'crs': str(src.crs),
                'nodata': src.nodata
            }
            
            # Spatial information
            bounds = src.bounds
            transform = src.transform
            
            spatial_info = {
                'bounds': {
                    'left': bounds.left,
                    'bottom': bounds.bottom,
                    'right': bounds.right,
                    'top': bounds.top
                },
                'pixel_size_x': transform.a,
                'pixel_size_y': abs(transform.e),
                'width_coords': bounds.right - bounds.left,
                'height_coords': bounds.top - bounds.bottom
            }
            
            return {'basic': basic_info, 'spatial': spatial_info}
            
    except Exception as e:
        print(f"Error processing {raster_path}: {e}")
        return None

# Test our function
result = explore_raster_detailed(elevation_file)
if result:
    print("✅ Function worked! Here's what we got:")
    print(f"Basic info keys: {list(result['basic'].keys())}")
    print(f"Spatial info keys: {list(result['spatial'].keys())}")

## 🧪 Step 8: Testing with Different Raster Files

Let's test our understanding with the other sample file (satellite imagery):

In [ ]:
# Test with the Landsat sample file
landsat_file = data_dir / 'landsat_sample.tif'

if landsat_file.exists():
    print("🛰️ LANDSAT SAMPLE ANALYSIS")
    print("=" * 40)
    
    with rasterio.open(landsat_file) as src:
        print(f"Dimensions: {src.width} x {src.height} pixels")
        print(f"Bands: {src.count}")
        print(f"Data type: {src.dtypes[0]}")
        print(f"CRS: {src.crs}")
        
        # Multi-band files might have different data types per band
        if src.count > 1:
            print(f"\nBand data types:")
            for i in range(src.count):
                print(f"  Band {i+1}: {src.dtypes[i]}")
        
        print(f"\nExtent: {src.bounds.left:.6f} to {src.bounds.right:.6f} (lon)")
        print(f"        {src.bounds.bottom:.6f} to {src.bounds.top:.6f} (lat)")
else:
    print("🔍 Landsat sample not found - that's okay, we'll focus on elevation data")

## ⚠️ Step 9: Error Handling and Edge Cases

Professional code needs to handle errors gracefully. Let's see what happens with common problems:

In [ ]:
def safe_raster_exploration(raster_path):
    """
    Example of robust error handling for raster files.
    This shows the kind of error handling your function should include.
    """
    print(f"🔍 Attempting to load: {raster_path}")
    
    # Check if file exists first
    if not Path(raster_path).exists():
        print(f"❌ File does not exist: {raster_path}")
        return None
    
    try:
        with rasterio.open(raster_path) as src:
            # Try to read basic properties
            info = {
                'filename': Path(raster_path).name,
                'width': src.width,
                'height': src.height,
                'count': src.count,
                'crs': str(src.crs)
            }
            print(f"✅ Successfully loaded {info['filename']}")
            return info
            
    except rasterio.RasterioIOError as e:
        print(f"❌ Rasterio error: {e}")
        return None
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        return None

# Test with good file
result1 = safe_raster_exploration(elevation_file)

# Test with missing file
result2 = safe_raster_exploration(data_dir / 'nonexistent_file.tif')

print(f"\nResults: Good file = {result1 is not None}, Missing file = {result2 is not None}")

## 🎯 Step 10: Your Implementation Template

Based on everything we've learned, here's the structure your `load_and_explore_raster()` function should follow:

In [ ]:
def load_and_explore_raster_template(raster_path):
    """
    Template for your assignment function.
    This shows the structure and key elements you need to implement.
    """
    # TODO: Import necessary libraries (rasterio, pathlib, etc.)
    
    # TODO: Check if file exists
    # HINT: Use Path(raster_path).exists()
    
    # TODO: Open raster file safely with context manager
    # HINT: Use 'with rasterio.open(raster_path) as src:'
    
    # TODO: Extract basic metadata
    # HINT: src.width, src.height, src.count, src.crs, etc.
    
    # TODO: Get spatial extent information  
    # HINT: src.bounds gives you left, bottom, right, top
    
    # TODO: Calculate pixel size from transform
    # HINT: src.transform.a for X pixel size, abs(src.transform.e) for Y
    
    # TODO: Print a clear summary for the user
    # HINT: Format output nicely with dimensions, extent, CRS info
    
    # TODO: Return a dictionary with all the key information
    # HINT: Organize into logical groups (basic info, spatial info, etc.)
    
    # TODO: Handle errors appropriately
    # HINT: FileNotFoundError, RasterioIOError, general Exception
    
    pass

print("📝 This template shows the structure your function should follow.")
print("🎯 Remember to implement each TODO section in src/rasterio_basics.py")

## 📋 Step 11: Expected Output Format

Here's what your function should return and print:

In [ ]:
# Example of what your function should output
expected_output_example = {
    'filename': 'elevation_dem.tif',
    'width': 1024,
    'height': 768,
    'count': 1,
    'dtype': 'float32',
    'driver': 'GTiff',
    'crs': 'EPSG:4326',
    'nodata': -9999.0,
    'bounds': {
        'left': -120.5,
        'bottom': 35.0,
        'right': -119.5,
        'top': 36.0
    },
    'pixel_size_x': 0.001,
    'pixel_size_y': 0.001,
    'extent_width': 1.0,
    'extent_height': 1.0
}

print("📊 Your function should return a dictionary similar to this structure:")
print("Keys to include:")
for key, value in expected_output_example.items():
    print(f"  '{key}': {type(value).__name__} - {value}")

print("\n💡 Adapt the exact values based on your actual data files!")

## ✅ Step 12: Ready to Implement!

Now you have everything you need to implement the `load_and_explore_raster()` function in `src/rasterio_basics.py`.

### 🎯 Key Requirements:
1. **Open raster files safely** using context managers
2. **Extract comprehensive metadata** (dimensions, bands, CRS, etc.)
3. **Calculate spatial properties** (extent, pixel size)
4. **Print clear summary** for user understanding
5. **Return structured data** as a dictionary
6. **Handle errors gracefully** with appropriate messages

### 🧪 Test Your Implementation:
```bash
uv run pytest tests/test_rasterio_basics.py::test_load_and_explore_raster -v
```

### 🔧 Debugging Tips:
- **Start simple**: Get basic file opening working first
- **Print intermediate values**: See what rasterio actually returns
- **Test with both sample files**: Make sure it works with different data types
- **Check return format**: Make sure your dictionary matches expected structure

### 📚 Next Step:
Once this function is working, move on to [`02_function_calculate_raster_statistics.ipynb`](02_function_calculate_raster_statistics.ipynb) to learn about analyzing pixel values!

---

**Good luck! 🍀 Remember: every expert was once a beginner who kept practicing!**